In [1]:
import os
import sys

project_directory = os.path.abspath('..')
sys.path.append(project_directory)

In [2]:
import torch
import torch.nn as nn
from src.utils.common import set_seed

set_seed(42)

In [3]:
from src.dataloaders.dataloader import create_dataloaders
train_loader, val_loader, test_loader = create_dataloaders(data_root='../data')

In [4]:
train_dataset, val_dataset, test_dataset = create_dataloaders(data_root='../data', return_datasets=True)

In [5]:
from src.models.modern_cnn_v1 import ModernCNNv1

In [6]:
from torch.utils.data import Subset, DataLoader

train_subset = Subset(test_dataset, range(64))
val_subset   = Subset(val_dataset, range(64))

In [7]:
train_loader_small = DataLoader(
    train_subset,
    batch_size=16,
    shuffle=True
)

val_loader_small = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False
)

In [13]:
model = ModernCNNv1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 20

In [14]:
from torch.optim.lr_scheduler import CosineAnnealingLR

schduler_cosine = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=epochs,
    eta_min=1e-6
)

In [15]:
from src.train.trainer import fit, EarlyStopper

In [16]:
early_stopper = EarlyStopper(patience=10)

test_result = fit(
    model=model,
    train_loader=train_loader_small,
    val_loader=val_loader_small,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE, 
    epochs=epochs,
    early_stopper=early_stopper,
    scheduler=schduler_cosine
)

Epoch 1/20 | LR before scheduler: 0.00100000
Epoch 1/20 | LR after scheduler:  0.00096761
EPOCH 1/20 | Train Loss: 2.667130 Acc: 0.140625 | Val Loss: 2.305073 Acc: 0.098400 
Epoch 2/20 | LR before scheduler: 0.00096761
Epoch 2/20 | LR after scheduler:  0.00096725
EPOCH 2/20 | Train Loss: 2.160228 Acc: 0.343750 | Val Loss: 2.318030 Acc: 0.095800 
Epoch 3/20 | LR before scheduler: 0.00096725
Epoch 3/20 | LR after scheduler:  0.00096469
EPOCH 3/20 | Train Loss: 1.696048 Acc: 0.437500 | Val Loss: 2.408141 Acc: 0.095800 
Epoch 4/20 | LR before scheduler: 0.00096469
Epoch 4/20 | LR after scheduler:  0.00096551
EPOCH 4/20 | Train Loss: 1.496826 Acc: 0.500000 | Val Loss: 2.379513 Acc: 0.163000 
Epoch 5/20 | LR before scheduler: 0.00096551
Epoch 5/20 | LR after scheduler:  0.00096270
EPOCH 5/20 | Train Loss: 1.166576 Acc: 0.578125 | Val Loss: 2.475907 Acc: 0.131200 
Epoch 6/20 | LR before scheduler: 0.00096270
Epoch 6/20 | LR after scheduler:  0.00095464
EPOCH 6/20 | Train Loss: 0.883872 Acc: 0

In [12]:
test_result

{'train_loss': [2.8800772428512573,
  2.2940482795238495,
  1.9525835514068604,
  1.8222190737724304,
  1.6104963719844818,
  1.422735184431076,
  1.112450271844864,
  1.1328329145908356,
  0.9082093983888626,
  0.7246208935976028,
  0.5875372886657715],
 'val_loss': [2.3058158504486084,
  2.3242517311096194,
  2.3740992649078367,
  2.459983418273926,
  2.4447377017974854,
  2.3190065200805665,
  2.3594265174865723,
  2.5766389347076415,
  2.5590186653137206,
  2.4844641273498533,
  2.605450138092041],
 'train_acc': [0.140625,
  0.3125,
  0.375,
  0.359375,
  0.46875,
  0.484375,
  0.6875,
  0.625,
  0.78125,
  0.703125,
  0.890625],
 'val_acc': [0.0958,
  0.1242,
  0.1042,
  0.1154,
  0.1492,
  0.191,
  0.194,
  0.1558,
  0.15,
  0.1982,
  0.2046]}